Master AI S1

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive/')

In [ ]:
cd '/content/drive/MyDrive'

In [ ]:
# save the final model to file
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adagrad
import tensorflow as tf

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = mnist.load_data()
	# reshape dataset to have a single channel
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax',name='dense_1'))
	# compile model
	opt = Adagrad(learning_rate=0.1,initial_accumulator_value=0.1,epsilon=1e-07,weight_decay=None,clipnorm=None,use_ema=False, ema_momentum=0.99, ema_overwrite_frequency=None, jit_compile=True, name='Adagrad')
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model()
	# fit model
	model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)
 	# save model
	model.save('/content/drive/MyDrive/mnist/myModel1',save_format='h5')
	# model.save('final_model.h5')


# entry point, run the test harness
run_test_harness()

In [ ]:
# make a prediction for a new image.
from numpy import argmax
from tensorflow.keras.utils import load_img
#from keras.preprocessing.image import load_img
from tensorflow.keras.utils import img_to_array
from keras.models import load_model
from numpy import argmax

#load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, grayscale=True, target_size=(28, 28))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 1 channel
	img = img.reshape(1, 28, 28, 1)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img

# load an image and predict the class
def run_example():
	# load the image
	img = load_image('/content/drive/MyDrive/mnist/6.png')
	# load model
	model = load_model('/content/drive/MyDrive/mnist/myModelnew')
	# predict the class
	predict_value = model.predict(img)
	digit = argmax(predict_value)
	print(digit)


	_, _, testX, testY = load_dataset()
	testX, _ = prep_pixels(testX, testX)
	loss, accuracy = model.evaluate(testX, testY, verbose=0)
	print("Test Accuracy:", accuracy)

# entry point, run the example
run_example()
